# Sources
* Model architecture based on [Liu and Gong, 2019](https://bmcbioinformatics.biomedcentral.com/counter/pdf/10.1186/s12859-019-3199-1.pdf), [Github](https://github.com/Jiale-Liu/LSTM)
* Co-teaching loss function and training process adaptations are based on [Han et al., 2018](https://arxiv.org/abs/1804.06872), and official implementation on [Github](https://github.com/bhanML/Co-teaching)


In [1]:
import os
import sys
import csv
import random
import math
import numpy as np
from time import time
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torchtext.vocab import build_vocab_from_iterator

from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [2]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(1234)

In [3]:
def read_data(path):
    with open(path, "r") as csvfile:
        train_data = list(csv.reader(csvfile))[1:]  # skip col name
        sents, lbls = [], []
        for s, l in train_data:
            sents.append(s)
            lbls.append(l)
    return sents, lbls


def apply_random_masking(seq, num_tokens):
    """
    Mask `num_tokens` as 0 at random positions per sequence.
    """
    dist = torch.rand(seq.shape)
    m, _ = torch.topk(dist, num_tokens)
    return seq * (dist < m)


def regularized_auc(train_auc, dev_auc, threshold=0.0025):
    """
    Returns development AUC if overfitting is below threshold, otherwise 0.
    """
    return dev_auc if (train_auc - dev_auc) < threshold else 0


def gelu(x):
    """
    Facebook Research implementation of the gelu activation function.
    
    For information: OpenAI GPT's gelu is slightly different
    (and gives slightly different results):
    0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))
    """
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))


def trainable_model_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def total_model_params(model):
    return sum(p.numel() for p in model.parameters())

In [4]:
class CleavageDataset(Dataset):
    def __init__(self, seq, lbl):
        self.seq = seq
        self.lbl = lbl
    
    def __getitem__(self, idx):
        return self.seq[idx], self.lbl[idx]
    
    def __len__(self):
        return len(self.lbl)
    
class TrainBatch:
    def __init__(self, batch):
        ordered_batch = list(zip(*batch))
        seq = torch.tensor([encode_text(seq) for seq in ordered_batch[0]], dtype=torch.int64)
        self.seq = apply_random_masking(seq, num_tokens=1)
        self.lbl = torch.tensor([int(l) for l in ordered_batch[1]], dtype=torch.float)
        
    def pin_memory(self):
        self.seq = self.seq.pin_memory()
        self.lbl = self.lbl.pin_memory()
        return self
    
def train_wrapper(batch):
    return TrainBatch(batch)


class EvalBatch:
    def __init__(self, batch):
        ordered_batch = list(zip(*batch))
        self.seq = torch.tensor([encode_text(seq) for seq in ordered_batch[0]], dtype=torch.int64)
        self.lbl = torch.tensor([int(l) for l in ordered_batch[1]], dtype=torch.float)
        
    def pin_memory(self):
        self.seq = self.seq.pin_memory()
        self.lbl = self.lbl.pin_memory()
        return self
    
def eval_wrapper(batch):
    return EvalBatch(batch)

In [5]:
class BiLSTMAttention(nn.Module):
    def __init__(self, vocab_size, embedding_dim, rnn_size, hidden_size, num_heads, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embedding_dim,
        )
        
        self.dropout=nn.Dropout(dropout)
        
        self.lstm1 = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=rnn_size,
            bidirectional=True,
            batch_first=True
        )
        
        self.lstm2 = nn.LSTM(
            input_size=2*rnn_size,
            hidden_size=rnn_size,
            bidirectional=True,
            batch_first=True
        )
        
        self.lstm3 = nn.LSTM(
            input_size=2*rnn_size,
            hidden_size=rnn_size,
            bidirectional=True,
            batch_first=True
        )
        
        self.lstm4 = nn.LSTM(
            input_size=2*rnn_size,
            hidden_size=rnn_size,
            bidirectional=True,
            batch_first=True
        )
        
        self.lstm5 = nn.LSTM(
            input_size=2*rnn_size,
            hidden_size=rnn_size,
            bidirectional=True,
            batch_first=True
        )
        
        self.attention = nn.MultiheadAttention(
            embed_dim=rnn_size * 2,
            num_heads=num_heads,
            batch_first=True
        )
        
        self.fc1 = nn.Linear(rnn_size * 2, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)
        
    def forward(self, seq):
        # input shape: (batch_size, seq_len=10)
        embedded = self.dropout(self.embedding(seq))
        
        # input shape: (batch_size, seq_len, embedding_dim)
        out1, (hn1, cn1) = self.lstm1(embedded)
        out2, (hn2, cn2) = self.lstm2(out1, (hn1, cn1))
        out3, (hn3, cn3) = self.lstm3(out2, (hn2, cn2))
        out3, hn3, cn3 = [torch.add(i, j) for i, j in zip([out1, hn1, cn1], [out3, hn3, cn3])]
        
        out4, (hn4, cn4) = self.lstm4(out3, (hn3, cn3))
        out5, (hn5, cn5) = self.lstm5(out4, (hn4, cn4))
        out5 = torch.add(out3, out5)
        
        out, _ = self.attention(out5, out1, out5)
        
        # input shape: (batch_size, seq_len, 2*rnn_size4)
        pooled, _ = torch.max(out, dim=1)
        
        # input shape: (batch_size, hidden_size)
        out = self.dropout(gelu(self.fc1(pooled)))
        
        # input shape: (batch_size, hidden_size)
        # shape: (batch_size)
        return self.fc2(out).squeeze()

In [6]:
def loss_coteaching(y_1, y_2, t, forget_rate):
    criterion = nn.BCEWithLogitsLoss(reduction='none')
    
    loss_1 = criterion(y_1, t)
    ind_1_sorted = np.argsort(loss_1.data.cpu())
    loss_1_sorted = loss_1[ind_1_sorted]

    loss_2 = criterion(y_2, t)
    ind_2_sorted = np.argsort(loss_2.data.cpu())
    loss_2_sorted = loss_2[ind_2_sorted]

    remember_rate = 1 - forget_rate
    num_remember = int(remember_rate * len(loss_1_sorted))

    ind_1_update = ind_1_sorted[:num_remember]
    ind_2_update = ind_2_sorted[:num_remember]
    
    # exchange
    loss_1_update = criterion(y_1[ind_2_update], t[ind_2_update])
    loss_2_update = criterion(y_2[ind_1_update], t[ind_1_update])

    return torch.sum(loss_1_update)/num_remember, torch.sum(loss_2_update)/num_remember

In [7]:
def train(model1, optim1, model2, optim2, loader, forget_rate):
    epoch_loss1, num_correct1, total = 0, 0, 0
    epoch_loss2, num_correct2 = 0, 0
    preds1, preds2, lbls = [], [], []
    
    for batch in tqdm(loader, desc="Train: ", file=sys.stdout, unit="batches"):
        seq, lbl = batch.seq, batch.lbl
        seq, lbl = seq.to(device), lbl.to(device)
        
        scores1 = model1(seq)
        scores2 = model2(seq)

        loss1, loss2 = loss_coteaching(scores1, scores2, lbl, forget_rate)
        
        optim1.zero_grad()
        loss1.backward()
        optim1.step()
        
        optim2.zero_grad()
        loss2.backward()
        optim2.step()
        
        epoch_loss1 += loss1.item()
        epoch_loss2 += loss2.item()
        num_correct1 += ((scores1 > 0) == lbl).sum().item()
        num_correct2 += ((scores2 > 0) == lbl).sum().item()
        total += seq.shape[0]
        preds1.extend(scores1.detach().tolist())
        preds2.extend(scores2.detach().tolist())
        lbls.extend(lbl.detach().tolist())
        
    return (
        epoch_loss1 / total,
        epoch_loss2 / total,
        num_correct1 / total,
        num_correct2 / total,
        roc_auc_score(lbls, preds1),
        roc_auc_score(lbls, preds2)
    )

In [8]:
def evaluate(model1, model2, loader, criterion):
    epoch_loss1, num_correct1, total = 0, 0, 0
    epoch_loss2, num_correct2 = 0, 0
    preds1, preds2, lbls = [], [], []
    
    for batch in tqdm(loader, desc="Eval: ", file=sys.stdout, unit="batches"):
        seq, lbl = batch.seq, batch.lbl
        seq, lbl = seq.to(device), lbl.to(device)
        
        scores1 = model1(seq)
        scores2 = model2(seq)

        loss1 = criterion(scores1, lbl)
        loss2 = criterion(scores2, lbl)
        
        epoch_loss1 += loss1.item()
        epoch_loss2 += loss2.item()
        num_correct1 += ((scores1 > 0) == lbl).sum().item()
        num_correct2 += ((scores2 > 0) == lbl).sum().item()
        total += seq.shape[0]
        preds1.extend(scores1.detach().tolist())
        preds2.extend(scores2.detach().tolist())
        lbls.extend(lbl.detach().tolist())
        
    return (
        epoch_loss1 / total,
        epoch_loss2 / total,
        num_correct1 / total,
        num_correct2 / total,
        roc_auc_score(lbls, preds1),
        roc_auc_score(lbls, preds2)
    )

In [9]:
def test(model, loader, criterion):
    epoch_loss, num_correct, total = 0, 0, 0
    preds, lbls = [], []
    
    for batch in tqdm(loader, desc="Eval: ", file=sys.stdout, unit="batches"):
        seq, lbl = batch.seq, batch.lbl
        seq, lbl = seq.to(device), lbl.to(device)
        
        scores = model(seq)

        loss = criterion(scores, lbl)
        
        epoch_loss += loss.item()
        num_correct += ((scores > 0) == lbl).sum().item()
        total += seq.shape[0]
        preds.extend(scores.detach().tolist())
        lbls.extend(lbl.detach().tolist())
        
    return epoch_loss / total, num_correct / total, roc_auc_score(lbls, preds)

In [10]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# load train and dev data
train_seqs, train_lbl = read_data('../../data/n_train.csv')
dev_seqs, dev_lbl = read_data('../../data/n_val.csv')

# create vocab from train seqs
vocab = build_vocab_from_iterator(train_seqs, specials=['<UNK>'])
vocab.set_default_index(vocab['<UNK>'])
encode_text = lambda x: vocab(list(x))

In [11]:
NUM_EPOCHS = 20
BATCH_SIZE = 512
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 216
RNN_SIZE = 111
HIDDEN_SIZE = 150
NUM_HEADS = 1
DROPOUT = 0.5
LEARNING_RATE = 1e-4

NUM_GRADUAL = 10 # how many epochs for linear drop rate
NOISY_RATE = 0.2
FORGET_RATE = 0.2
EXPONENT = 1

# define drop rate schedule
rate_schedule = np.ones(NUM_EPOCHS)*FORGET_RATE
rate_schedule[:NUM_GRADUAL] = np.linspace(0, FORGET_RATE**EXPONENT, NUM_GRADUAL)

params = {
    "vocab_size": VOCAB_SIZE,
    "embedding_dim": EMBEDDING_DIM,
    "rnn_size": RNN_SIZE,
    "hidden_size": HIDDEN_SIZE,
    "num_heads": NUM_HEADS,
    "dropout": DROPOUT
}

model1 = BiLSTMAttention(**params).to(device)
model2 = BiLSTMAttention(**params).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer1 = optim.Adam(model1.parameters(), lr=LEARNING_RATE)
optimizer2 = optim.Adam(model2.parameters(), lr=LEARNING_RATE)

# create train and dev loader
train_data = CleavageDataset(train_seqs, train_lbl)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=train_wrapper, pin_memory=True, num_workers=10)

dev_data = CleavageDataset(dev_seqs, dev_lbl)
dev_loader = DataLoader(dev_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=eval_wrapper, pin_memory=True, num_workers=10)

In [12]:
start = time()
print("Starting Training.")
highest_val_auc = 0

# normal training loop
for epoch in range(1, NUM_EPOCHS + 1):
    model1.train()
    model2.train()
    train_loss1, train_loss2, train_acc1, train_acc2, train_auc1, train_auc2 = train(
        model1=model1,
        optim1=optimizer1,
        model2=model2,
        optim2=optimizer2,
        loader=train_loader,
        forget_rate=rate_schedule[epoch-1],
    )

    model1.eval()
    model2.eval()
    with torch.no_grad():
        val_loss1, val_loss2, val_acc1, val_acc2, val_auc1, val_auc2 = evaluate(
            model1=model1,
            model2=model2,
            loader=dev_loader,
            criterion=criterion
        )
    
    print(
        f"Model 1 Training:   [Epoch {epoch:2d}, Loss: {train_loss1:8.6f}, Acc: {train_acc1:.4f}, AUC: {train_auc1:.4f}]"
    )
    print(
        f"Model 1 Evaluation: [Epoch {epoch:2d}, Loss: {val_loss1:8.6f}, Acc: {val_acc1:.4f}, AUC: {val_auc1:.4f}]"
    )
    print(
        f"Model 2 Training:   [Epoch {epoch:2d}, Loss: {train_loss2:8.6f}, Acc: {train_acc2:.4f}, AUC: {train_auc2:.4f}]"
    )
    print(
        f"Model 2 Evaluation: [Epoch {epoch:2d}, Loss: {val_loss2:8.6f}, Acc: {val_acc2:.4f}, AUC: {val_auc2:.4f}]"
    )

    if val_auc1 > val_auc2:
        reg_auc = regularized_auc(train_auc1, val_auc1, threshold=0)
        model = model1.state_dict()
        print('saved model1')
    else:
        reg_auc = regularized_auc(train_auc2, val_auc2, threshold=0)
        model = model2.state_dict()
        print('saved model2')
    if reg_auc > highest_val_auc:
        highest_val_auc = reg_auc
        path = f"../../params/n_term/BiLSTMAttention_coteaching/auc{reg_auc:.4f}_epoch{epoch}.pt"
        torch.save(model, path)

print("Finished Training.")
train_time = (time() - start) / 60
print(f"Training took {train_time} minutes.")

Starting Training.
Eval: 100%|██████████████████████████████████████████████████████| 281/281 [00:01<00:00, 192.65batches/s]
Model 1 Training:   [Epoch  1, Loss: 0.000887, Acc: 0.8173, AUC: 0.6463]
Model 1 Evaluation: [Epoch  1, Loss: 0.000811, Acc: 0.8219, AUC: 0.7544]
Model 2 Training:   [Epoch  1, Loss: 0.000886, Acc: 0.8168, AUC: 0.6473]
Model 2 Evaluation: [Epoch  1, Loss: 0.000801, Acc: 0.8251, AUC: 0.7580]
saved model2
Eval: 100%|██████████████████████████████████████████████████████| 281/281 [00:01<00:00, 188.63batches/s]
Model 1 Training:   [Epoch  2, Loss: 0.000693, Acc: 0.8229, AUC: 0.7406]
Model 1 Evaluation: [Epoch  2, Loss: 0.000903, Acc: 0.8285, AUC: 0.7667]
Model 2 Training:   [Epoch  2, Loss: 0.000691, Acc: 0.8231, AUC: 0.7422]
Model 2 Evaluation: [Epoch  2, Loss: 0.000895, Acc: 0.8288, AUC: 0.7690]
saved model2
Eval: 100%|██████████████████████████████████████████████████████| 281/281 [00:01<00:00, 195.89batches/s]
Model 1 Training:   [Epoch  3, Loss: 0.000575, Acc: 0

In [13]:
test_path = '../../data/n_test.csv'
test_seqs, test_lbls = read_data(test_path)

test_data = CleavageDataset(test_seqs, test_lbls)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, collate_fn=eval_wrapper, pin_memory=True, num_workers=10)

# load best model, evaluate on test set
best_model = sorted(
    [f for f in os.listdir("../../params/n_term/BiLSTMAttention_coteaching/") if f.endswith(".pt")],
    reverse=True,
)[0]
print("Loaded model: ", best_model)
model1.load_state_dict(torch.load('../../params/n_term/BiLSTMAttention_coteaching/' + best_model))
model1.eval()

with torch.no_grad():
    test_loss, test_acc, test_auc = test(model1, test_loader, criterion)
print(
    f"Test Set Performance: Loss: {test_loss:.6f}, Acc: {test_acc:.4f}, AUC: {test_auc:.4f}"
)
print(
    f"Total model params: {total_model_params(model1)}, trainable model params: {trainable_model_params(model1)}"
)

Loaded model:  auc0.7728_epoch4.pt
Eval: 100%|██████████████████████████████████████████████████████| 281/281 [00:00<00:00, 358.20batches/s]
Test Set Performance: Loss: 0.001312, Acc: 0.8313, AUC: 0.7748
Total model params: 1718233, trainable model params: 1718233
